In [43]:
import json
import pandas as pd
import numpy as np
import random
import string
from string import punctuation
import torch.nn.functional as F
from gensim.parsing.preprocessing import remove_stopwords

In [44]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

In [45]:
filename = "dataset/web_science_dataset.jsonl"
json_data = []
with open(filename) as f:
    json_data = f.readlines()
json_data_list = []
for item in json_data:
    json_data_list.append(json.loads(item))
df_ = pd.DataFrame(json_data_list)

In [46]:
## Load data

In [47]:
df = pd.read_csv('cleaned_data.csv',delimiter=",")
df.head()
df.shape

(957, 7)

In [48]:
map_ = dict(zip(df_['questionId'].astype(int),df_['answer']))
df['answer'] = df['questionId'].map(map_)

map_cat = dict(zip(df_['questionId'].astype(int),df_['categoryId']))
df['categoryId'] = df['questionId'].map(map_cat)

df.head()

,Question,Answer URL,Answer Label,Question Rating,Answer Quality,Factual,questionId,answer,categoryId
0,does water have a memory as claimed in homeopathy,https://skeptics.stackexchange.com/questions/2#27,no,1.666667,2.333333,1,2,No\n\nWater forms strong intermolecular forces...,3
1,does chamomile help you to relax,https://skeptics.stackexchange.com/questions/3...,na,2.000000,2.000000,1,3,There is a website by the NIH about Chamomile ...,0
2,are there benefits to the eca stack for bodybu...,https://skeptics.stackexchange.com/questions/2...,yes,1.666667,2.000000,1,22,It appears that the combination of ephedrine a...,0
3,can positive thinking provide an improved outc...,https://skeptics.stackexchange.com/questions/2...,yes,2.666667,2.666667,1,26,To add to Krzysztof's answer. There were also ...,2
4,are vegetables good for me,https://skeptics.stackexchange.com/questions/3...,yes,2.000000,2.333333,1,32,"\n From a young age, most people are told tha...",0


In [49]:
df['answer'].apply(lambda x: np.mean(len(x)))

0      1328.0
1       599.0
2      1912.0
3      2044.0
4      2535.0
        ...  
952    2051.0
953    2216.0
954     404.0
955    3757.0
956    1149.0
Name: answer, Length: 957, dtype: float64

In [50]:
df.head()

,Question,Answer URL,Answer Label,Question Rating,Answer Quality,Factual,questionId,answer,categoryId
0,does water have a memory as claimed in homeopathy,https://skeptics.stackexchange.com/questions/2#27,no,1.666667,2.333333,1,2,No\n\nWater forms strong intermolecular forces...,3
1,does chamomile help you to relax,https://skeptics.stackexchange.com/questions/3...,na,2.000000,2.000000,1,3,There is a website by the NIH about Chamomile ...,0
2,are there benefits to the eca stack for bodybu...,https://skeptics.stackexchange.com/questions/2...,yes,1.666667,2.000000,1,22,It appears that the combination of ephedrine a...,0
3,can positive thinking provide an improved outc...,https://skeptics.stackexchange.com/questions/2...,yes,2.666667,2.666667,1,26,To add to Krzysztof's answer. There were also ...,2
4,are vegetables good for me,https://skeptics.stackexchange.com/questions/3...,yes,2.000000,2.333333,1,32,"\n From a young age, most people are told tha...",0


#### Preprocessing the dataset

In [51]:
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    return (' '.join(lemmatizer.lemmatize(w) for w in word_tokenize(text)))

In [52]:
def preprocess_text(df):    

    # remove leading/trailing spaces
    df = df.str.strip()
    
    # convert to lowercase
    df = df.str.lower()
    
    df = df.replace(to_replace ='http\S+', value = '', regex = True)
    
    # remove punctuation
    translator = str.maketrans('', '', string.punctuation) 
    df = df.str.translate(translator)
    
    # remove non-alphanumeric characters
    df = df.replace(to_replace ='\s*[^A-Za-z0-9]+\s*', value = ' ', regex = True)
    
    # remove digits
    translator = str.maketrans('', '', string.digits) 
    df = df.str.translate(translator)
    
    df = df.str.strip()
    
    #lemmatize
    #df = df.apply(lemmatize_text)
    
    return df

In [53]:
df['Question'] = preprocess_text(df['Question'])
df['answer'] = preprocess_text(df['answer'])
df['Answer Quality'] = ((df['Answer Quality']-1)/(3-1))

In [54]:
for i, row in df.iterrows():
    text = remove_stopwords(row['answer'])
    #print(text)
    new_text = " ".join(text.split(" ")[:75])
    #print(new_text)
    df.at[i,'answer'] = new_text

mean_answer_lengths = df['answer'].apply(lambda x: np.mean(len(x)))
mean_answer_lengths

0      637.0
1      344.0
2      658.0
3      537.0
4      547.0
       ...  
952    643.0
953    571.0
954    260.0
955    540.0
956    627.0
Name: answer, Length: 957, dtype: float64

In [55]:
mean_answer_lengths = df['Question'].apply(lambda x: np.mean(len(x)))
mean_answer_lengths

0      49.0
1      32.0
2      52.0
3      64.0
4      26.0
       ... 
952    70.0
953    45.0
954    51.0
955    34.0
956    66.0
Name: Question, Length: 957, dtype: float64

In [56]:
#df['answer'].values

In [57]:
np.unique(df['Answer Quality'].values)

array([0.        , 0.16666667, 0.25      , 0.33333333, 0.5       ,
       0.66666667, 0.75      , 0.83333333, 0.875     , 1.        ])

In [58]:
df.dtypes

Question            object
Answer URL          object
Answer Label        object
Question Rating    float64
Answer Quality     float64
Factual              int64
questionId           int64
answer              object
categoryId           int64
dtype: object

In [59]:
def read_file(filename):
    with open(filename) as f:
        data = f.readlines()
        data = [int(i.strip()) for i in data ]
    return data

In [60]:
train_ids = read_file('quality_prediction_data/training_ids.txt')
test_ids = read_file('quality_prediction_data/testing_ids.txt')

In [61]:
train_data = df.loc[df['questionId'].isin(train_ids)]
test_data = df.loc[df['questionId'].isin(test_ids)]

In [62]:
from sklearn.model_selection import train_test_split

train_data, valid_data = train_test_split(df,stratify=df['categoryId'].values, test_size=0.2)
train_data.shape
valid_data.shape
valid_data.head()

,Question,Answer URL,Answer Label,Question Rating,Answer Quality,Factual,questionId,answer,categoryId
889,is burning ha of grassland more environmentall...,https://skeptics.stackexchange.com/questions/4...,na,1.333333,0.666667,1,42318,african grass species pennisetum purpureum mea...,1
168,do women prefer taller men,https://skeptics.stackexchange.com/questions/4...,yes,1.500000,0.500000,1,4409,based wikipedia page physical attractiveness r...,4
683,is the asshole effect a phenomenon correlated ...,https://skeptics.stackexchange.com/questions/2...,yes,2.666667,0.833333,1,26805,yes piff compiled evidence correlation wealths...,4
169,are chemically treatedheavy foods bad for you,https://skeptics.stackexchange.com/questions/4...,yes,1.666667,0.166667,1,4441,hear lot people talk bad processed foods havin...,0
159,did an xray burst interrupt radio signals shor...,https://skeptics.stackexchange.com/questions/4...,na,1.333333,0.000000,1,4226,solar storms site opinion reliability says sun...,3


In [63]:
unique_labels = set(train_data['Answer Quality'])
unique_labels

{0.0,
 0.16666666649999995,
 0.25,
 0.33333333349999994,
 0.5,
 0.6666666665000001,
 0.75,
 0.8333333334999999,
 0.875,
 1.0}

In [64]:
unique_labels_test = set(test_data['Answer Quality'])

In [65]:
unique_labels_test

{0.0,
 0.16666666649999995,
 0.25,
 0.33333333349999994,
 0.5,
 0.6666666665000001,
 0.75,
 0.8333333334999999,
 1.0}

In [66]:
label_map = {v:k for k,v in enumerate(unique_labels)}
label_map

{0.8333333334999999: 0,
 1.0: 1,
 0.33333333349999994: 2,
 0.0: 3,
 0.5: 4,
 0.6666666665000001: 5,
 0.75: 6,
 0.16666666649999995: 7,
 0.25: 8,
 0.875: 9}

### Load pretrained word embeddings: word2vec

In [67]:
from gensim.models import KeyedVectors
from gensim.utils import tokenize
import pandas as pd
import numpy as np

In [68]:
EMBEDDING_FILE = "C:\\Users\\Reen\\Desktop\\web science\\WordEmbeddings\\GoogleNews-vectors-negative300.bin.gz"
word_to_vec_model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True,limit=50000)

In [69]:
pretrained_embeddings = np.concatenate([word_to_vec_model.vectors, np.zeros(shape=(1,300))], axis=0) #last vector padding

## Model

In [70]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook as tqdm
from torch import nn
from torch.optim import Adam
from typing import List, Tuple

In [71]:
seed = 1000
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

num_labels = len(label_map)
batch_size = 64
lr = 2e-4
lstm_dim = 25
n_epochs = 200
#device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

### Loading data into the model

In [72]:
class ClassificationDatasetReader():
    
    def __init__(self,df):
        self.df = df
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        row = self.df.values[idx]
        input_seq,seq_lens = text_to_batch_bilstm([row[7]]) #sending a list of one row
        label = row[4]
        return input_seq, seq_lens, label

In [73]:
def collate_batch_bilstm(input_data: Tuple) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    input_ids = [i[0][0] for i in input_data]
    seq_lens = [i[1][0] for i in input_data]
    labels = [i[2] for i in input_data]

    max_length = max([len(i) for i in input_ids])

    input_ids = [(i + [50000] * (max_length - len(i))) for i in input_ids]

    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens), torch.tensor(labels)

In [74]:
def text_to_batch_bilstm(text: List) -> Tuple[List, List]:
    oov_word_token = 500000 #oov token
    
    sentences = []
    for t in text:
        t = t.lower()
        t = ''.join([c for c in t if c not in punctuation])
    sentences.append(t)
        
    tokens = [ list(tokenize(t)) for t in sentences]
    #print(tokens)
    
    input_ids = [[word_to_vec_model.vocab[token].index for token in sentence if token in word_to_vec_model] for sentence in tokens]
    #print(input_ids)
    
    #input_ids = [tokenizer.encode_ids_with_eos(t) for t in text]

    return input_ids, [len(ids) for ids in input_ids]

In [75]:
# Create the dataset readers and read data
train_dataset = ClassificationDatasetReader(train_data)
train_dl = DataLoader(train_dataset, batch_size=len(train_data), shuffle=True, collate_fn=collate_batch_bilstm)

valid_dataset = ClassificationDatasetReader(valid_data)
valid_dl = DataLoader(valid_dataset, batch_size=len(valid_data), collate_fn=collate_batch_bilstm)

test_dataset = ClassificationDatasetReader(test_data)
test_dl = DataLoader(test_dataset, batch_size=len(test_data), collate_fn=collate_batch_bilstm)
valid_data.shape

(192, 9)

In [76]:
train_data.shape

(765, 9)

## Creating the model

In [77]:
# Define the model
class LSTMNetwork(nn.Module):

    def __init__(self, pretrained_embeddings: torch.tensor, lstm_dim: int, dropout_prob: float = 0.1, n_classes: int =9):
        
        super(LSTMNetwork, self).__init__()
        
        #LSTM input
        self.input_size = pretrained_embeddings.shape[1] #features in input
        self.hidden_size = 50
        self.num_layers = 1 #default 
        self.n_classes = n_classes
        
        self.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1)
        self.lstm = nn.LSTM(self.input_size,self.hidden_size,self.num_layers,batch_first=True,dropout=dropout_prob,bidirectional=True)
        self.ff = (nn.Linear(2*self.hidden_size,1))
        
        
    def forward(self, inputs, input_lens, labels = None):
        
        # Get embeddings (b x sl x edim)
        embeds = self.embeddings(inputs)
        lstm_out, hidden = self.lstm(embeds)
        
        # Get the last output for classification (b x 2*lstm_dim)
        ff_in = lstm_out.gather(1, input_lens.view(-1,1,1).expand(lstm_out.size(0), 1, lstm_out.size(2)) - 1).squeeze()
        
        # Get logits (b x 2)
        regression = torch.sigmoid(self.ff(ff_in).view(-1,1))
        
        outputs = (regression,)
        if labels is not None:
            # Xentropy loss
            loss_fn = nn.MSELoss()
            labels.to(device, dtype= torch.float)
            loss = loss_fn(regression, labels.unsqueeze_(1))
            outputs = (loss,) + outputs

        return outputs

In [78]:
# Create the model
model = LSTMNetwork(
    pretrained_embeddings=torch.FloatTensor(pretrained_embeddings), 
    lstm_dim=lstm_dim, 
    dropout_prob=0, 
    n_classes=len(label_map)
  ).to(device)

## Training 

In [79]:
def accuracy(logits, labels):
    #return torch.sum(torch.argmax(logits, dim=-1) == labels).type(torch.float) / float(labels.shape[0])
    rmse = torch.sqrt(torch.sum((logits - labels)**2)/float(labels.shape[0]))
    return rmse

In [80]:
def train(model, train_dl, valid_dl, optimizer, n_epochs, device):
    
    losses = []
    best_acc = 0.0
    best_loss = None
  
    for ep in range(n_epochs):
        loss_epoch = []
        
        for batch in tqdm(train_dl):
            
            model.train()
            optimizer.zero_grad()
            
            batch = tuple(t.to(device) for t in batch)
            input_ids = batch[0]
            seq_lens = batch[1]
            labels = batch[2]

            loss, logits = model(input_ids, seq_lens, labels=labels)
            losses.append(loss.item())
            loss_epoch.append(loss.item())
      
            #print((loss.dtype))
            #loss = loss.float()
            loss.to(device, dtype= torch.float)
            loss.backward()
            #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
        #gc.collect()
    
        # Evaluate
        model.eval()
        with torch.no_grad():
            for batch in valid_dl:

                batch = tuple(t.to(device) for t in batch)
                input_ids = batch[0]
                seq_lens = batch[1]
                labels = batch[2]

                val_loss, logits = model(input_ids, seq_lens, labels=labels)
                acc = accuracy(logits, labels)

                print(f'RMSE: {acc}, train loss: {sum(loss_epoch) / len(loss_epoch)}',"val_loss: ",val_loss.item())
                best_model = model.state_dict()

                if(best_loss is None):
                    best_model = model.state_dict()
                    best_loss = val_loss.item()
                elif(best_loss>val_loss.item()):
                    best_model = model.state_dict()
                    best_loss = val_loss.item()
                #gc.collect()

    model.load_state_dict(best_model)
    return model, losses

In [ ]:
# Create the optimizer
optimizer = Adam(model.parameters(), lr=lr)

# Train
model, losses = train(model, train_dl, valid_dl, optimizer, n_epochs, device)


RMSE: 0.26242679357528687, train loss: 0.07667653262615204 val_loss:  0.06886781752109528



RMSE: 0.26158714294433594, train loss: 0.07623285055160522 val_loss:  0.06842783093452454



RMSE: 0.2607627511024475, train loss: 0.07579819858074188 val_loss:  0.06799721717834473



RMSE: 0.25995364785194397, train loss: 0.07537256926298141 val_loss:  0.06757589429616928



RMSE: 0.2591596245765686, train loss: 0.07495584338903427 val_loss:  0.06716370582580566


...


RMSE: 0.2404911369085312, train loss: 0.0598783940076828 val_loss:  0.05783598870038986



RMSE: 0.24054613709449768, train loss: 0.05977347493171692 val_loss:  0.05786244198679924



RMSE: 0.24060361087322235, train loss: 0.059666913002729416 val_loss:  0.05789009854197502



RMSE: 0.24066375195980072, train loss: 0.05955862998962402 val_loss:  0.05791904032230377



RMSE: 0.24072667956352234, train loss: 0.05944856256246567 val_loss:  0.057949334383010864


In [ ]:
def test_model(model = model, test_dl = test_dl, device = device):
    model.eval()
      with torch.no_grad():    
        for batch in test_dl:
            batch = tuple(t.to(device) for t in batch)
            input_ids = batch[0]
            seq_lens = batch[1]
            labels = batch[2]
        
            _test_loss, logits = model(input_ids, seq_lens, labels=labels)
            acc = accuracy(logits, labels)
        
          print(f'RMSE: {acc}','test_loss:',_test_loss.item())

In [587]:
test_model()


RMSE: 0.16327278316020966 test_loss: 0.02665800042450428
